<div class="alert alert-block alert-info">
<p style="font-size:24px;text-align:center"><b>Test the initialization of the 800nm laser pulse and Ti:Sapphire crystal</b>
</div>

In [ ]:
set = 'old' 
# Options: 'old', 'new', 'function'
prop_type = 'n0n2_srw' 
# Options: 'gain_calc' 'abcd_lct' 'n0n2_lct' 'n0n2_srw'
set_pump_type = 'left'
# Options: 'dual' 'left' 'right'

num_laser_slices = 10
num_crystal_slices = 10

gain = 1
radial_n2 = 1

### Imports, Files, Crystal/Laser Params

In [ ]:
import sys, copy
import numpy as np
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import crystal
import rslaser.utils.srwl_uti_data as srwutil

import scipy.constants as const
from scipy import interpolate
from scipy import special

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

In [ ]:
if set == 'old':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220218')
    ccd_name = 'photon_count_pump_off.txt'
    wfs_name = 'phase_pump_off.txt'
elif set == 'new':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220519')
    ccd_name = 'cut_photon_count_seed_laser.txt'
    wfs_name = 'phase_pump_off_zeroed.txt'    

files = PKDict(
    meta = os.path.join(package_data_dir, 'meta_data.dat'),
    ccd = os.path.join(package_data_dir, ccd_name),
    wfs = os.path.join(package_data_dir, wfs_name)
)

z       = np.array([ 0.0,  0.5,  1.0, 1.5, 2.0, 2.5])/100.0
n2_plot = np.array([30.5, 18.3, 10.4, 5.9, 3.3, 1.9])
plot_fit = interpolate.splrep(z, n2_plot)

In [ ]:
w0 = 1.64e-3

params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = num_laser_slices,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / np.sqrt(2.),
    sigx_waist  = w0,
    sigy_waist  = w0,
)

L_cryst = 0.025
new_z = (L_cryst /num_crystal_slices) *(np.arange(num_crystal_slices)+0.5)

crystal_params = PKDict(
    length      = L_cryst, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = interpolate.splev(new_z, plot_fit).tolist(),
    population_inversion = PKDict(
        n_cells=64,
        mesh_extent=0.01,  # [m]
        crystal_alpha=120.0,  # [1/m], 1.2 1/cm
        pump_waist=1.43e-3,  # [m]
        pump_wavelength=532.0e-9,  # [m]
        pump_energy=0.035,  # [J], pump laser energy onto the crystal
        pump_type=set_pump_type,
    )
)

L_drift = 0.5  # [m]

cell_dx = (2.0 *crystal_params.population_inversion.mesh_extent)/crystal_params.population_inversion.n_cells

***
## Total number of excited states in full crystal, as a function of the number of crystal slices

In [ ]:
total_n_excited_states_i = np.zeros(crystal_params.nslice)
total_n_excited_states   = np.zeros(crystal_params.nslice)
crystal_params_copy = copy.deepcopy(crystal_params)

for num_c_slices in np.arange(crystal_params.nslice):

    num_slices = num_c_slices+1
    new_z = (L_cryst /num_slices) *(np.arange(num_slices)+0.5)
    crystal_params_copy.nslice = num_c_slices+1
    crystal_params_copy.n0 = [1.76 for _ in range(num_slices)]
    crystal_params_copy.n2 = interpolate.splev(new_z, plot_fit).tolist()
    e_crystal = crystal.Crystal(crystal_params_copy)
    cell_volume = cell_dx**2.0*(crystal_params_copy.length/crystal_params_copy.nslice)
    
    if set == 'function':
        thisPulse = pulse.LaserPulse(params)
    else:
        thisPulse = pulse.LaserPulse(params, files)
    
    for crystal_index_i in np.arange(e_crystal.nslice):
        total_n_excited_states_i[num_c_slices] += np.sum(e_crystal.slice[crystal_index_i].pop_inversion_mesh *cell_volume)
    
    if (num_c_slices < crystal_params.nslice):
        print('\nPropagating ', len(thisPulse.slice), 'laser slices through ', e_crystal.nslice,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type, gain, radial_n2)

    for crystal_index_p in np.arange(e_crystal.nslice):
        total_n_excited_states[num_c_slices] += np.sum(e_crystal.slice[crystal_index_p].pop_inversion_mesh *cell_volume)

n_c_slice = np.arange(1,crystal_params.nslice + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_c_slice,total_n_excited_states_i,'k',label='after initialization')
plt.plot(n_c_slice,total_n_excited_states,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Number Excited States')
ax.set_xlabel(r'Number of Crystal Slices')

***
## Number of excited states as a function of longitudinal distance through the crystal

In [ ]:
e_crystal = crystal.Crystal(crystal_params)
cell_volume = cell_dx**2.0*(crystal_params.length/crystal_params.nslice)
n_excited_states_slice_i = np.zeros(e_crystal.nslice)
n_excited_states_slice = np.zeros(e_crystal.nslice)

if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

for c_slice_index in np.arange(e_crystal.nslice):
    n_excited_states_slice_i[c_slice_index] = np.sum(e_crystal.slice[c_slice_index].pop_inversion_mesh * cell_volume)

print('\nPropagating ', thisPulse.nslice, 'laser slices through ', e_crystal.nslice,' crystal slices')
thisPulse = e_crystal.propagate(thisPulse, prop_type, gain, radial_n2)

for c_slice_index in np.arange(e_crystal.nslice):
    n_excited_states_slice[c_slice_index] = np.sum(e_crystal.slice[c_slice_index].pop_inversion_mesh * cell_volume)

#Assume all crystals have the same length
z_c_slice = e_crystal.slice[0].length *(np.arange(e_crystal.nslice)+0.5)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(z_c_slice,n_excited_states_slice_i,'k',label='after initialization')
plt.plot(z_c_slice,n_excited_states_slice,'--k',label='after propagation')
# plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Number Excited States')
ax.set_xlabel(r'Distance from Crystal Face [m]')
ax.set_title('      Longitudinal Distribution of Excited States')


***
## Single pass, number of photons in laser as a function of number of slices in crystal

In [ ]:
total_n_photons_i = np.zeros(crystal_params.nslice)
total_n_photons   = np.zeros(crystal_params.nslice)

for num_c_slices in np.arange(crystal_params.nslice):

    num_slices = num_c_slices+1
    new_z = (L_cryst /num_slices) *(np.arange(num_slices)+0.5)
    crystal_params_copy.nslice = num_c_slices+1
    crystal_params_copy.n0 = [1.76 for _ in range(num_slices)]
    crystal_params_copy.n2 = interpolate.splev(new_z, plot_fit).tolist()
    e_crystal = crystal.Crystal(crystal_params_copy)
    cell_volume = cell_dx**2.0*(crystal_params_copy.length/crystal_params_copy.nslice)
    
    if set == 'function':
        thisPulse = pulse.LaserPulse(params)
    else:
        thisPulse = pulse.LaserPulse(params, files)
    
    for laser_index_i in np.arange(thisPulse.nslice):
        total_n_photons_i[num_c_slices] += np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh)
    
    if (num_c_slices < crystal_params.nslice):
        print('\nPropagating ', thisPulse.nslice, 'laser slices through ', e_crystal.nslice,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type, gain, radial_n2)

    for laser_index_p in np.arange(thisPulse.nslice):
        total_n_photons[num_c_slices] += np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh)

n_c_slice = np.arange(1,crystal_params.nslice + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_c_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_c_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Crystal Slices')


***
## Total number of photons in full laser pulse, as a function of the number of laser slices

In [ ]:
total_n_photons_i = np.zeros(params.nslice)
total_n_photons   = np.zeros(params.nslice)
params_copy = copy.deepcopy(params)

for num_l_slices in np.arange(params.nslice):
    
    e_crystal = crystal.Crystal(crystal_params)
    params_copy.nslice = num_l_slices+1
    
    if set == 'function':
        thisPulse = pulse.LaserPulse(params_copy)
    else:
        thisPulse = pulse.LaserPulse(params_copy, files)
    
    for laser_index_i in np.arange(thisPulse.nslice):
        total_n_photons_i[num_l_slices] += np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh)
    
    if (num_l_slices < params.nslice):
        print('\nPropagating ', thisPulse.nslice, 'laser slices through ', e_crystal.nslice,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type, gain, radial_n2)

    for laser_index_p in np.arange(thisPulse.nslice):
        total_n_photons[num_l_slices] += np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh)

n_l_slice = np.arange(1,params.nslice + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_l_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Laser Slices in Pulse')

## Number of photons as a function of longitudinal distance through the laser pulse
***

In [ ]:
if thisPulse.slice[0].ds == thisPulse.slice[-1].ds:
    fig_x = thisPulse.slice[0].ds *(np.arange(thisPulse.nslice) + 0.5)
else:
    print('Slices are different lengths!')

photons_in_slice = np.zeros((thisPulse.nslice))
for laser_index in np.arange(thisPulse.nslice):
        photons_in_slice[laser_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d.mesh))

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(fig_x,photons_in_slice,'k')
box = ax.get_position()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')